In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import multinomial, geom, boltzmann, dlaplace
from scipy.stats import poisson
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data
import torch.optim as optim

In [2]:
def zipf_dist(alpha, N, size):
    x = np.arange(1, N+1, dtype='float')
    weights = x ** (-alpha)
    weights /= weights.sum()
    bounded_zipf = stats.rv_discrete(name='bounded_zipf', values=(x, weights))
    data = np.reshape(np.array(bounded_zipf.rvs(size=size)), [-1, 1])
    return data, weights

In [3]:
def harmonic(n, alpha):
    a = 0
    for i in range(1, n):
        a += 1/i**alpha
    return a

In [4]:
def zipf_entropy(alphabet, alpha):
    p = np.arange(1, alphabet, dtype='float')**(-alpha)
    c = harmonic(alphabet, alpha)
    H_zipf = -(1/c)*np.sum(p*np.log(p/c))
    return H_zipf

In [5]:
# Other utils
def convert_to_one_hot(y, dict_size=None):
    if dict_size is None:
        dict_size = np.unique(y).shape[0]
    y_hot = np.eye(dict_size)[y.astype('int32')]
    return y_hot

def make_one_hot(y, dims, dict_size=None):
    y_hot = []
    for i in range(dims):
        y_hot.append(convert_to_one_hot(y[:, i], dict_size))
    return y_hot

def make_rnn_data(time_series, sequence_length):
    seq_lst = []
    time_series = np.array(time_series)
    for i in range((len(time_series)-sequence_length+1)):
        seq_lst.append(time_series[i:(i+sequence_length), :])
    return np.array(seq_lst)

In [6]:
class ModelBasicClassification(nn.Module):
    def __init__(self, input_shape, class_size):
        super(ModelBasicClassification, self).__init__()
        self.lstm = nn.LSTM(input_size=input_shape[1], hidden_size=50, batch_first=True)
        self.fc = nn.Linear(50, class_size)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x)
        return x

In [7]:
# Generate data from zipf
alpha = 1
alphabet = 10**5
size = 1000
p = 1/alphabet
data, _ = zipf_dist(alpha, alphabet, size)
H_true = zipf_entropy(alphabet, alpha)

# Convert symbols to binary representation
dims = 25 # size of the binary representation, chosen by the dectated by the alphabet size
vf = np.vectorize(np.binary_repr)
data=data.astype(np.int64)
data = vf(data, width=dims)
lst = []
for i in data:
    lst.append([int(j) for j in i[0]])
data = np.array(lst)

# Prepare RNN input and output
data_shift = np.hstack([data[:, 1:], np.zeros(shape=(len(data), 1))])
data = np.expand_dims(data, axis=2)
seq_lst = []
for i in data_shift:
    y = np.reshape(i, (-1, 1))
    seq_lst.append(make_one_hot(y, 1, 2))
data_shift = np.array(seq_lst)
data_shift = np.squeeze(data_shift, axis=1)

In [8]:
seq_length = 25
epochs = 2000
batch_size = 64
bins = 2

data = torch.from_numpy(data).float()
data_shift = torch.from_numpy(data_shift).long()

model = ModelBasicClassification([seq_length, 1], 2)
opt = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
    opt.zero_grad()
    output = model(data)
    loss = criterion(output.view(-1, 2), torch.argmax(data_shift.view(-1, 2), dim=1))
    loss.backward()
    opt.step()

output = model(data)
# CE = output.gather(2, data_shift.view(-1,1)).squeeze(2)
# CE=output.gather(1, data_shift.view(-1)).squeeze(1)

# CE = -torch.log(CE.max(dim=1)[0]).sum(dim=0).mean().item()
CE = output*data_shift
CE = (-torch.log(torch.sigmoid(CE.sum(dim=2)))).sum(dim=1).mean().item()
#print(CE)

y = data[:, 0]
p_1 = (y.sum().item() + 1e-5) / float(size)
H_1 = -torch.sum(torch.tensor([p_1, 1-p_1]) * torch.log(torch.tensor([p_1, 1-p_1])))
CE = CE + H_1.item()
#print(CE)

In [9]:
print('H estimated', CE)
print('H true', H_true)

H estimated 8.115833466677515
H true 7.967998491428407
